In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from time import sleep

load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

base_url = (
    "https://api.themoviedb.org/3/discover/movie"
    "?include_adult=false"
    "&include_video=false"
    "&language=en-US"
    "&sort_by=primary_release_date.asc"
    "&with_original_language=da"
    "&primary_release_date.gte=2020-01-01"
    "&primary_release_date.lte=2024-12-31"
)

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}

movie_ids = []

# Loop through all pages to find the movie_ids (adjust max page if needed)
for page in range(1, 501):
    url = f"{base_url}&page={page}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Status code: {response.status_code} Text: {response.text}")
    
    # Extract movie_results from the response
    movie_results = response.json().get("results") 
    
    # Stop if we have reached the last page
    if not movie_results:
        break

    # Extract movie_ids from the movie_results
    movie_ids.extend([movie["id"] for movie in movie_results])
    
    sleep(0.02)  # Just to be sure we don't request too many requests

print(f"Number of movie ids found: {len(movie_ids)}")

Number of movie ids found: 931


In [2]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from time import sleep

load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}

all_movies = []

# Loop through all movie_ids and request movie data
for movie_id in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?language=en-US&append_to_response=credits"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Status code: {response.status_code} Text: {response.text}")
    
    # Extract the movie data from the response
    movie = response.json()

    # Simplify dictionaries to list of ids
    movie["genre_ids"] = [genre["id"] for genre in movie["genres"]]
    movie["spoken_languages"] = [language["iso_639_1"] for language in movie["spoken_languages"]]
    movie["production_company_ids"] = [company["id"] for company in movie["production_companies"]]
    movie["production_countries"] = [country["iso_3166_1"] for country in movie["production_countries"]]
    movie["collection_id"] = movie.get("belongs_to_collection").get("id") if movie["belongs_to_collection"] else None
    movie["cast_person_ids"] = [cast_member["id"] for cast_member in movie["credits"]["cast"]]
    movie["cast_credit_ids"] = [cast_member["credit_id"] for cast_member in movie["credits"]["cast"]]
    movie["crew_person_ids"] = [crew_member["id"] for crew_member in movie["credits"]["crew"]]
    movie["crew_credit_ids"] = [crew_member["credit_id"] for crew_member in movie["credits"]["crew"]]
    del movie['genres']
    del movie['production_companies']
    del movie["belongs_to_collection"]   
    del movie["credits"]
    
    all_movies.append(movie)
    sleep(0.02)  # Just to be sure we don't request too many requests

df_movies = pd.DataFrame(all_movies)
print(f"Total movies collected: {len(df_movies)}")

output_folder = os.path.join(os.getcwd(), "movie_data")
output_path = os.path.join(output_folder, "alt.csv")
df_movies.to_csv(output_path, index=False)

Total movies collected: 931


In [3]:
df_movies

,adult,backdrop_path,budget,homepage,id,imdb_id,origin_country,original_language,original_title,overview,...,video,vote_average,vote_count,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
0,False,None,0,,1389683,None,[DK],da,Den gamle mand og skoven,,...,False,0.0,0,"[99, 10770]",[119],NaN,[],[],"[1114991, 1114991, 1114991, 5066576, 5066578]","[673b21f4f741eb0428b62a1e, 673b2207f741eb0428b..."
1,False,None,0,,1275299,None,[DK],da,"Badabing og Bang - Hurra, årtiet er slut!",,...,False,0.0,0,[99],[119],NaN,[91568],[661ea6efd18fb90131c9c55c],[],[]
2,False,None,0,,1176704,tt26752729,[DK],da,Stime,"During the summer, the relationship between tw...",...,False,0.0,0,[18],[166089],NaN,"[3018873, 2120409, 2974764, 4265498, 4265499, ...","[64ff095fffc9de0eded38e34, 64ff096cffc9de0edf6...","[2653084, 2653084, 2653084, 208175, 135949]","[64ff09aaffc9de0ee3c53cfc, 64ff09b02dffd800e3d..."
3,False,/3tdNtSbcMliHlWnxt5fVuiL6QmO.jpg,0,,660040,None,[DK],da,Grin til gavn 2019,"In keeping with tradition, the popular comedy ...",...,False,6.0,1,[35],[20632],NaN,"[88359, 88355, 1949847, 2040551, 2208065, 1556...","[5e16d23e11386c0015c92dd1, 5e16d2470cb33500170...",[],[]
4,False,/w2T6QLixu8X6C5FlfpKDglIFcJ4.jpg,0,,661167,None,[DK],da,Et langsomt mord,,...,False,0.0,0,[99],[639],NaN,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,False,/MSP35R6y0ypmf3zU1kyUw1P4It.jpg,0,,1407803,None,[DK],da,Verdensmænd - Bobos surprise,,...,False,10.0,1,[35],[758],NaN,"[1828049, 234058, 1355004, 88356, 2208065]","[676d44614ca2fbbfca614283, 676d44784ca2fbbfca6...",[],[]
927,False,/2iVVzCEnX4QMEeyd3e52juYcsH8.jpg,0,,1410638,None,[DK],da,Gud bevare Danmark,,...,False,0.0,0,[],[],NaN,[],[],[],[]
928,False,/ej5uQVMJTzq8nZBCbf6pUMNF6jl.jpg,0,,1409581,None,[DK],da,Frank Hvam - Nobody,Frank Hvam has lived in New Zealand with his f...,...,False,6.0,1,[35],[758],NaN,[89973],[67724668d8dc22e6c6927586],"[89973, 5520795, 563909, 5227452, 4263643, 516...","[68650819af3c4b2566414b34, 686508769febd451ea8..."
929,False,None,0,,1412034,None,[DK],da,"Dan Andersen - Nedsat Hørelse, Nedsat Sædkvali...",,...,False,0.0,0,[35],[],NaN,[149860],[677903662b097b15a274ab1e],[],[]
